## Preprocessing: one-hotエンコーディング・欠損値処理

one-hot encodingと欠損値処理を学ぶため、ローン審査結果データを用います。

In [ ]:
# import sample data: Loan screening data for classification 
import pandas as pd

df = pd.read_csv('./data/av_loan_u6lujuX_CVtuZ9i.csv',header=0)
X = df.iloc[:,:-1]           # 最終列以前を特徴量X
X = X.drop('Loan_ID',axis=1) # 1列目はID情報のため特徴量から削除
y = df.iloc[:,-1]            # 最終列を正解データ

# check the shape
print('X shape: (%i,%i)' %X.shape)

# ローン審査でNOとなったサンプルを1（正例）へ変換
class_mapping = {'N':1, 'Y':0}
y = y.map(class_mapping)
print('--------------------')
print(y.value_counts())
X.join(y).head()

上記例えば、LoanAmountの1行目に欠損値を確認できます。<b>この欠損をLoanAmount列の平均値で置き換えることを欠損値補完、GenderやMarriedのようなカテゴリ変数を0/1のバイナリ変数に変換することをone-hotエンコーディング</b>と言います。

本講座では一連の前処理の統一のため、<b>(1)まずone-hotエンコードをしてカテゴリ変数の欠損をフラグ変数化し解決した上で、(2)残った連続変数の欠損値を平均値で置き換えることとします</b>。それではone-hotエンコードの実施です。オプションのdummy_na=Trueとしておきましょう。これにより欠損が入っていたというのが情報化されます。

In [ ]:
ohe_columns = ['Dependents',
               'Gender',
               'Married',
               'Education',
               'Self_Employed',
               'Property_Area']
X_new = pd.get_dummies(X,
                       dummy_na=True,
                       columns=ohe_columns)
X_new.head()

上記まででカテゴリ変数の数量化と欠損処理は終了です。<b>次に連続変数の欠損を平均値で置き換えます。この処理はsklearnのImputerクラスで実現できます</b>。処理の正常確認のため、LoanAmountの基礎統計量を確認しておきましょう。平均値が146.412162であることが確認して下さい。

In [ ]:
X_new.describe()

それでは連続変数の欠損値の平均値補完の実行です。preporcessingクラスからImputerを読み込みます。Imputerクラスのメソッドtransfomrを適用することで、LoanAmountの欠損値（1行目など）を、NaNから平均値（146.412162）に置き換えることができます。

In [ ]:
from sklearn.preprocessing import Imputer

# インピュータークラスの実体化
imp = Imputer(missing_values='NaN', # 欠損値NaNを
              strategy='mean',      # 平均値で置換
              axis=0)               # 列方向に処理

# 各特徴量の平均値を学習
imp.fit(X_new)

# 学習済みのImputerを適用し, X_newの欠損値を置き換える.
X_new_columns = X_new.columns.values
X_new = pd.DataFrame(imp.transform(X_new),
                     columns=X_new_columns)
# 結果表示
X_new.head()

以上で、one-hotエンコーディングと欠損値補完の処理は終了です。この時点でも特徴量の次元数が小さければ、そのままアルゴリズムに投入しても構いません。ただし、実務においては数百・数千次元を超えることがしばしばありますので、その際は以下の次元圧縮を行います。

## Preprocessing: 次元圧縮（RFE&PCA)

さて、特徴量が元の11次元が26次元まで増加しました。<br><b>ここではRFEを使って、予測に役立つと判断された上位10変数に絞り込むこととします。</b>

In [ ]:
from sklearn.feature_selection import RFE
from sklearn.ensemble import RandomForestClassifier

# 特徴量因子の重要度を推定する分類器をRandomForestClassifierに設定
# 最終的に残す特徴量を10に設定
# 1回のstepで削除する次元数は5%ずつとする
selector = RFE(estimator=RandomForestClassifier(random_state=0),
               n_features_to_select=10,
               step=.05)
selector.fit(X_new,y)
print('Done normally')

RFEをfitすることで、26変数のうちどの変数を残すかが決定されました。<br><b>残された変数の確認は"support_"属性を呼び出すことで可能です。</b><br>Trueが採用された変数の場所を表しています。

In [ ]:
print(selector.support_)

fitまでで選択すべき変数を決めることができたので、実際にデータの絞り込み処理をしましょう。<br>Imputerと同様にデータの変換はtransformでできます。

In [ ]:
# 26次元を10次元を圧縮
X_new_selected=selector.transform(X_new)
X_new_selected=pd.DataFrame(X_new_selected,
                            columns=X_new_columns[selector.support_])
print('---------------------------------------')
print('X shape after RFE:', X_new_selected.shape)
print('---------------------------------------')
print(X_new_selected.dtypes)
X_new_selected.head()

またRFEの亜種としてRFECVというライブラリが用意されており、これは選択する特徴量の個数を自動で判定してくれます。計算負荷は大きくなりますが、データ件数が少なく、ハイパーパラメータを減らしたい場合には有用です。

In [ ]:
from sklearn.feature_selection import RFECV
from sklearn.ensemble import RandomForestClassifier

selector = RFECV(estimator=RandomForestClassifier(random_state=0),
                 step=0.05)
X_new_selected = selector.fit_transform(X_new, y)
print(X_new_selected.shape)

RFEによる特徴量次元の絞り込みは以上で終了です。予測モデリングでは、このRFE済み特徴量Xを、交叉検証にかけベストモデルを選択することになります。さて最後に、<b>PCAによる次元圧縮の方法を確認しましょう。</b>最もシンプルな実装はPCAをパイプラインに組み込む方法です。X_new（RFEをする前の26次元の特徴量）を対象にPCAをさせる方法は以下です。

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import f1_score

# パイプラインにPCAを埋め込めば自動的に次元圧縮してくれる
clf = Pipeline([('scl', StandardScaler()),
                ('reduct', PCA(n_components=10,random_state=1)),
                ('clf', GradientBoostingClassifier(random_state=1))])

# 学習時に自動的にPCA処理が施される
clf.fit(X_new, y)
print('Normally done')

学習器clfの実態はパイプラインですから、<b>これを学習済みモデルとして保存しておけば、学習済みscl、学習済みreduct(PCA)、学習済みclf（モデル）の3つが学習状態で保存されます</b>。